# Analyse reanalysis data at point - station

Code correct as of Summer 2023.

Analyse data from processed ERA-Interim/ERA5/ERA5-Land csv files. The currently available daily data (all means unless otherwise stated) are:

* MSLP (msl) or surface pressure (sp - ERA5-land)
* 2m air temperature -- daily mean (t2m/mean_t2m), maximum (mx2t/max_t2m) and minimum (mn2t/min_t2m)
* soil temperature on 4 levels (stl1, stl2, stl3, stl4)
* 10m wind field components (u10, v10)
* 2m dew point temperature (d2m)
* daily total precipitation (tp)
* snow depth (sd) and snow density (rsn)

The variable codes shown in brackets above are the names of the directories in which the data for that variable are held on the bas_climate gws.

* ERA-Interim data are available January 1979 to August 2019
* ERA5 data are available January 1940 to present
* ERA5-Land data are available January 1950 to present

## Imports - always run

In [ ]:
from datetime import datetime
from pathlib import Path
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# Analyse data

## Sub-daily data

In [ ]:
# Here, and throughout, edit variable names as necessary
# Load processed csv file and define variables to be analysed
df = pd.read_table(f'/home/users/clelland/era5/stations_pandas_files/{station}/{station}e53h.csv', sep=";", index_col='time')
df = df[['u10', 'v10', 't2m', 'mn2t', 'mx2t', 'd2m', 'msl']]

# Convert Kelvin temp values to Celsius
def edit_temp_values(x):
    return x - 273.15
df['t2m'] = df['t2m'].apply(edit_temp_values)
df['mn2t'] = df['mn2t'].apply(edit_temp_values)
df['mx2t'] = df['mx2t'].apply(edit_temp_values)
df['d2m'] = df['d2m'].apply(edit_temp_values)

# Convert Pascal pressure values to hPa/mb (1 hPa = 1 mb)
def edit_msl_values(x):
    return x / 100
df['msl'] = df['msl'].apply(edit_msl_values)

# find wspeed and bearing
pi = np.pi
df['wspeed'] = np.sqrt((df['u10']**2) + (df['v10']**2))

def find_bearing(x, y):
    return (np.arctan2(x, y) + pi)
df['bearing'] = ((find_bearing(df['u10'], df['v10']))*180/pi).round()

df.index = pd.to_datetime(df.index)

# (If necessary) edit the time period to deal with missing data from the met stations
df = df['1966-01-01 01:00:00':]
df = df.loc[(df.index < '1977-10-31 16:00:00') | (df.index > '1977-11-30 13:00:00')]

df

In [ ]:
# read in met station data master file
dfsrok = pd.read_table("/gws/nopw/j04/bas_climate/users/clelland/met_data/SROK8Cdata.txt", sep=";", header=None) # <-- CHANGE AS NECESSARY

# key variables from SROK8C main table:
# 25, 26: wind direction (from), speed, 28:tp, 29:stl1? - treat separately, 34:t2m, 37:mn2t, 38:mx2t, 43:d2m, 45:msl

In [ ]:
# get relevant rows from table - check row index for relevant years then use iloc
dfstation = dfsrok.copy()
#dfstation = dfstation[dfstation[0] == station]
#dfstation = dfstation[dfstation[1] >= 1979]
#dfstation = dfstation[dfstation[1] <= 2019]
dfstation = dfstation.iloc[3:160475]

# set datetime wrt timezone and if applicable use dfcsv index as index to ensure continuity
# column 11 refers to timezone - take timezone off the local hour
dfstation = dfstation.rename(columns={1:'year', 2:'month', 3:'day', 4:'hour'})
def edit_time_values(x):
    return x + 1
dfstation['hour'] = dfstation['hour'].apply(edit_time_values)
dfstation['time'] = pd.to_datetime(dfstation[['year', 'month', 'day', 'hour']])
dfstation = dfstation.set_index(df.index)

# select columns to use and rename
dfstation = dfstation[[25, 26, 34, 37, 38, 43, 45]]
dfstation = dfstation.rename(columns={25:'bearing1', 26:'wspeed1', 34:'t2m1', 37:'mn2t1', 38:'mx2t1', 43:'d2m1', 45:'msl1'})

# fill in missing values with NaN
dfstation = dfstation.replace('     ', np.nan, regex = True)

# convert values to float
dfstation['bearing1'] = pd.to_numeric(dfstation['bearing1'],errors = 'coerce')
dfstation['wspeed1'] = pd.to_numeric(dfstation['wspeed1'],errors = 'coerce')
dfstation = dfstation.astype(float)
                         
# work out u10 and v10 values
# note the negative signs - the bearing gives the location of where the wind comes from, hence actual wind direction is 180o opposite
pi = np.pi
dfstation['theta1'] = (dfstation['bearing1']*pi)/180

for i in dfstation['bearing1']:
    def find_u10_values(z):
        if 0 < i <= 90:
            x = np.sin(z)
        elif 90 < i <= 180:
            x = np.cos(z - (pi/2))
        elif 180 < i <= 270:
            x = np.sin(z - pi)
        else:
            x = np.cos(z - ((3*pi)/2))
        return -x * dfstation['wspeed1']
dfstation['u101'] = (find_u10_values(dfstation['theta1'])).round(2)

for i in dfstation['bearing1']:
    def find_v10_values(z):
        if 0 < i <= 90:
            x = np.cos(z)
        elif 90 < i <= 180:
            x = np.sin(z - (pi/2))
        elif 180 < i <= 270:
            x = np.cos(z - pi)
        else:
            x = np.sin(z - ((3*pi)/2))
        return -x * dfstation['wspeed1']
dfstation['v101'] = (find_v10_values(dfstation['theta1'])).round(2)

# round theta column
dfstation['theta1'] = dfstation['theta1'].round(2)
                         
dfstation

In [ ]:
# Check data quality: column max, min, number of NaNs
#dfstation['tp1'].max()
#dfstation['tp1'].min()
#dfstation['bearing1'].isna().sum()

In [ ]:
# merge the datasets
result = pd.concat([df, dfstation], axis=1)
result = result[['u10', 'u101', 'v10', 'v101', 'wspeed', 'wspeed1', 'bearing', 'bearing1', 'msl', 'msl1', 't2m', 't2m1', 'mn2t', 'mn2t1', 'mx2t', 'mx2t1', 'd2m', 'd2m1']]
result.index = pd.to_datetime(result.index)
result = result['1979-01-01 01:00:00':'2019-08-31 22:00:00']
result

In [ ]:
# find correlations
print(result['t2m'].corr(result['t2m1']))
print(result['mn2t'].corr(result['mn2t1']))
print(result['mx2t'].corr(result['mx2t1']))
print(result['d2m'].corr(result['d2m1']))
print(result['msl'].corr(result['msl1']))
print(result['u10'].corr(result['u101']))
print(result['v10'].corr(result['v101']))
print(result['wspeed'].corr(result['wspeed1']))
print(result['bearing'].corr(result['bearing1']))

In [ ]:
# find variance ratio
print((result['t2m'].var())/(result['t2m1'].var()))
print((result['mn2t'].var())/(result['mn2t1'].var()))
print((result['mx2t'].var())/(result['mx2t1'].var()))
print((result['d2m'].var())/(result['d2m1'].var()))
print((result['msl'].var())/(result['msl1'].var()))
print((result['u10'].var())/(result['u101'].var()))
print((result['v10'].var())/(result['v101'].var()))
print((result['wspeed'].var())/(result['wspeed1'].var()))
print((result['bearing'].var())/(result['bearing1'].var()))

In [ ]:
# find RMSE, ignoring certain rows due to NaN values
resultt2m = result[result['t2m1'].notna()]
resultmn2t = result[result['mn2t1'].notna()]
resultmn2t = resultmn2t[resultmn2t['mn2t'].notna()]
resultmx2t = result[result['mx2t1'].notna()]
resultmx2t = resultmx2t[resultmx2t['mx2t'].notna()]
resultd2m = result[result['d2m1'].notna()]
resultmsl = result[result['msl1'].notna()]
resultu10 = result[result['u101'].notna()]
resultv10 = result[result['v101'].notna()]
resultwsp = result[result['wspeed1'].notna()]
resultbea = result[result['bearing1'].notna()]

print(np.sqrt(mean_squared_error(resultt2m['t2m1'], resultt2m['t2m'])))
print(np.sqrt(mean_squared_error(resultmn2t['mn2t1'], resultmn2t['mn2t'])))
print(np.sqrt(mean_squared_error(resultmx2t['mx2t1'], resultmx2t['mx2t'])))
print(np.sqrt(mean_squared_error(resultd2m['d2m1'], resultd2m['d2m'])))
print(np.sqrt(mean_squared_error(resultmsl['msl1'], resultmsl['msl'])))
print(np.sqrt(mean_squared_error(resultu10['u101'], resultu10['u10'])))
print(np.sqrt(mean_squared_error(resultv10['v101'], resultv10['v10'])))
print(np.sqrt(mean_squared_error(resultwsp['wspeed1'], resultwsp['wspeed'])))
print(np.sqrt(mean_squared_error(resultbea['bearing1'], resultbea['bearing'])))

## Daily data

In [ ]:
# Here, and throughout, edit variable names as necessary
# Load processed csv file and define variables to be analysed
dfdaily = pd.read_table(f'/home/users/clelland/era5/stations_pandas_files/{station}/{station}e5pdaily.csv', sep=";", index_col='time')
dfdaily = dfdaily[['stl2', 'stl3', 'stl4', 'msl', 'sd', 'rsn', 't2m', 'mx2t', 'mn2t', 'tp', 'u10', 'v10']]

# Convert Kelvin temp values to Celsius
def edit_temp_values(x):
    return x - 273.15
dfdaily['stl2'] = dfdaily['stl2'].apply(edit_temp_values)
dfdaily['stl3'] = dfdaily['stl3'].apply(edit_temp_values)
dfdaily['stl4'] = dfdaily['stl4'].apply(edit_temp_values)
dfdaily['mn2t'] = dfdaily['mn2t'].apply(edit_temp_values)
dfdaily['mx2t'] = dfdaily['mx2t'].apply(edit_temp_values)
dfdaily['t2m'] = dfdaily['t2m'].apply(edit_temp_values)

# sd values need to be converted to snowfall in m and then * 100 to convert to cm - see https://confluence.ecmwf.int/pages/viewpage.action?pageId=155325734
# actual snow depth = (sd*1000)/rsn
def edit_sd_values(x, y):
    return ((x*1000)/y) * 100
dfdaily['sdadj'] = edit_sd_values(dfdaily['sd'], dfdaily['rsn'])

# Convert m to mm for precipitation
def edit_tp_values(x):
    return x * 1000
dfdaily['tp'] = dfdaily['tp'].apply(edit_tp_values)

# Convert Pascal pressure values to hPa/mb (1 hPa = 1 mb)
def edit_msl_values(x):
    return x / 100
dfdaily['msl'] = dfdaily['msl'].apply(edit_msl_values)

# find wspeed
pi = np.pi
dfdaily['wspeed'] = np.sqrt((dfdaily['u10']**2) + (dfdaily['v10']**2))

dfdaily = dfdaily[['stl2', 'stl3', 'stl4', 'sdadj', 't2m', 'mx2t', 'mn2t', 'tp', 'u10', 'v10', 'wspeed', 'msl']]

dfdaily.index = pd.to_datetime(dfdaily.index)
dfdaily = dfdaily.set_index(dfdaily.index.to_period('D'))

dfdaily

In [ ]:
# read in met station data - CHANGE AS NECESSARY
dfsnow = pd.read_table("/gws/nopw/j04/bas_climate/users/clelland/met_data/SNOWdata.txt", sep=";", header=None)
dfsoil = pd.read_table("/gws/nopw/j04/bas_climate/users/clelland/met_data/TPGdata.txt", sep=";", header=None)
dftemp = pd.read_table("/gws/nopw/j04/bas_climate/users/clelland/met_data/TTTRdata.txt", sep=";", header=None)

In [ ]:
# Snow depth
# get relevant rows from table - check row index for relevant years then use iloc
dfstationsnow = dfsnow.copy()
#dfstationsnow = dfstationsnow[dfstationsnow[0] == station]
#dfstationsnow = dfstationsnow[dfstationsnow[1] >= 1979]
#dfstationsnow = dfstationsnow[dfstationsnow[1] <= 2019]
dfstationsnow = dfstationsnow.iloc[24625:39966]

# set datetime wrt timezone and use dfcsvdaily index as index to ensure continuity
dfstationsnow = dfstationsnow.rename(columns={1:'year', 2:'month', 3:'day'})
dfstationsnow['time'] = pd.to_datetime(dfstationsnow[['year', 'month', 'day']])
dfstationsnow = dfstationsnow.set_index(dfstationsnow.time)

# select columns to use and rename
dfstationsnow = dfstationsnow[[4]]
dfstationsnow = dfstationsnow.rename(columns={4:'sd1'})

# fill in missing values with NaN
dfstationsnow = dfstationsnow.mask(np.isclose(dfstationsnow.values, 9999))

# convert values to float
dfstationsnow = dfstationsnow.astype(float)

# define tieme period for analysis
dfstationsnow = dfstationsnow['1979-01-01':'2019-08-31']
dfstationsnow = dfstationsnow.set_index(dfdaily.index)

dfstationsnow

In [ ]:
# Soil temps
# get relevant rows from table - check row index for relevant years then use iloc
dfstationsoil = dfsoil.copy()
#dfstationsoil = dfstationsoil[dfstationsoil[0] == 23678]
#dfstationsoil = dfstationsoil[dfstationsoil[1] >= 1979]
#dfstationsoil = dfstationsoil[dfstationsoil[1] <= 2020]
dfstationsoil = dfstationsoil.iloc[744:7750]

# select which columns to use and rename
dfstationsoil = dfstationsoil[[8, 9, 11, 13, 15]]
dfstationsoil = dfstationsoil.rename(columns={8:'20cm', 9:'40cm', 11:'80cm', 13:'160cm', 15:'320cm'})

# convert values to float
dfstationsoil = dfstationsoil.astype(float)

# fill in missing dfstationsoil with NaN and drop rows with all NaN
dfstationsoil = dfstationsoil.mask(np.isclose(dfstationsoil.values, 999.9))
dfstationsoil = dfstationsoil.dropna(how='all')

# data finishes in October 1997 - get relevant rows from dfdaily
dfdailyreduced = dfdaily.copy()
dfdailyreduced = dfdailyreduced['1977-01-01':'1997-10-31']
dfdailyreduced = dfdailyreduced.loc[(dfdailyreduced.index < '1977-11-01') | (dfdailyreduced.index > '1977-11-30')]
#dfdailyreduced = dfdailyreduced.loc[:'1997-10-31']

# set datetime wrt timezone and use dfcsvdaily index as index to ensure continuity
dfstationsoil = dfstationsoil.set_index(dfdailyreduced.index)

# estimate 60cm temps
dfstationsoil['60cm'] = (dfstationsoil['40cm'] + dfstationsoil['80cm'])/2

dfstationsoil = dfstationsoil[['20cm', '40cm', '60cm', '80cm', '160cm', '320cm']]
dfstationsoil

In [ ]:
# Temperature and precipitation variables
# get relevant rows from table - check row index for relevant years then use iloc
dfstationtemp = dftemp.copy()
#dfstationtemp = dfstationtemp[dfstationtemp[0] == station]
#dfstationtemp = dfstationtemp[dfstationtemp[1] >= 1979]
#dfstationtemp = dfstationtemp[dfstationtemp[1] <= 2020]
dfstationtemp = dfstationtemp.iloc[24837:40178]

# set datetime wrt timezone and use dfcsv index as index to ensure continuity
dfstationtemp = dfstationtemp.rename(columns={1:'year', 2:'month', 3:'day'})
dfstationtemp['time'] = pd.to_datetime(dfstationtemp[['year', 'month', 'day']])
dfstationtemp = dfstationtemp.set_index(dfstationtemp.time)

# select columns to use and rename
dfstationtemp = dfstationtemp[[5, 6, 7, 8]]
dfstationtemp = dfstationtemp.rename(columns={5:'mn2t1', 6:'t2m1', 7:'mx2t1', 8:'tp1'})

# convert values to float
dfstationtemp = dfstationtemp.replace('     ', np.nan, regex = True)
dfstationtemp = dfstationtemp.astype(float)

# set time period
dfstationtemp = dfstationtemp['1979-01-01':'2019-08-31']
dfstationtemp = dfstationtemp.set_index(dfdaily.index)

dfstationtemp

In [ ]:
# Wind speed - from sub-daily data
dfstationwind = result.copy()
dfstationwind = dfstationwind[['wspeed1', 'msl1']]
dfstationwind.index = pd.to_datetime(dfstationwind.index)
dfstationwind = dfstationwind.resample('D').mean()
#dfstationwind = dfstationwind.set_index(dfdaily.index)

dfdailywind = dfdaily.copy()
dfdailywind = dfdailywind['1979-01-01':'2019-08-31']
dfstationwind = dfstationwind.set_index(dfdailywind.index)

dfstationwind

In [ ]:
# Check data quality: column max, min, number of NaNs
#dfstationsnow['tp'].max()
#dfstationsnow['tp'].min()
#dfstationsnow['tp'].isna().sum()

In [ ]:
# merge the datasets
resultdaily = pd.concat([dfdaily, dfstationsnow, dfstationtemp, dfstationwind], axis=1)
resultdaily = resultdaily[['sdadj', 'sd1', 'SD', 'RSN', 't2m', 't2m1', 'mx2t', 'mx2t1', 'mn2t', 'mn2t1', 'tp', 'tp1', 'wspeed', 'wspeed1', 'msl', 'msl1']]
resultdaily

In [ ]:
resultdaily['tpbias'] = resultdaily['tp1'] - resultdaily['tp']
tpbias = (resultdaily['tpbias'].sum())/(resultdaily['tpbias'].notnull().sum())
resultdaily['t2mbias'] = resultdaily['t2m1'] - resultdaily['t2m']
t2mbias = (resultdaily['t2mbias'].sum())/(resultdaily['t2mbias'].notnull().sum())
resultdaily['mn2tbias'] = resultdaily['mn2t1'] - resultdaily['mn2t']
mn2tbias = (resultdaily['mn2tbias'].sum())/(resultdaily['mn2tbias'].notnull().sum())
resultdaily['mx2tbias'] = resultdaily['mx2t1'] - resultdaily['mx2t']
mx2tbias = (resultdaily['mx2tbias'].sum())/(resultdaily['mx2tbias'].notnull().sum())
resultdaily['sdbias'] = resultdaily['sd1'] - resultdaily['sdadj']
sdbias = (resultdaily['sdbias'].sum())/(resultdaily['sdbias'].notnull().sum())
resultdaily['wspbias'] = resultdaily['wspeed1'] - resultdaily['wspeed']
wspbias = (resultdaily['wspbias'].sum())/(resultdaily['wspbias'].notnull().sum())
resultdaily['mslbias'] = resultdaily['msl1'] - resultdaily['msl']
mslbias = (resultdaily['mslbias'].sum())/(resultdaily['mslbias'].notnull().sum())
print(tpbias)
print(t2mbias)
print(mn2tbias)
print(mx2tbias)
print(sdbias)
print(wspbias)
print(mslbias)

In [ ]:
# find correlations
print(resultdaily['tp'].corr(resultdaily['tp1']))
print(resultdaily['t2m'].corr(resultdaily['t2m1']))
print(resultdaily['mn2t'].corr(resultdaily['mn2t1']))
print(resultdaily['mx2t'].corr(resultdaily['mx2t1']))
print(resultdaily['sdadj'].corr(resultdaily['sd1']))
print(resultdaily['wspeed'].corr(resultdaily['wspeed1']))
print(resultdaily['msl'].corr(resultdaily['msl1']))

In [ ]:
# find variance ratio
print((resultdaily['tp'].var())/(resultdaily['tp1'].var()))
print((resultdaily['t2m'].var())/(resultdaily['t2m1'].var()))
print((resultdaily['mn2t'].var())/(resultdaily['mn2t1'].var()))
print((resultdaily['mx2t'].var())/(resultdaily['mx2t1'].var()))
print((resultdaily['sdadj'].var())/(resultdaily['sd1'].var()))
print((resultdaily['wspeed'].var())/(resultdaily['wspeed1'].var()))
print((resultdaily['msl'].var())/(resultdaily['msl1'].var()))

In [ ]:
# find rmse - actual then predicted
resultdailytp = resultdaily[resultdaily['tp'].notna()]
resultdailytp = resultdailytp[resultdailytp['tp1'].notna()]
resultdailyt2m = resultdaily[resultdaily['t2m1'].notna()]
resultdailymn2t = resultdaily[resultdaily['mn2t'].notna()]
resultdailymn2t = resultdailymn2t[resultdailymn2t['mn2t1'].notna()]
resultdailymx2t = resultdaily[resultdaily['mx2t'].notna()]
resultdailymx2t = resultdailymx2t[resultdailymx2t['mx2t1'].notna()]
resultdailysd = resultdaily[resultdaily['sd1'].notna()]
resultdailywsp = resultdaily[resultdaily['wspeed1'].notna()]
resultdailymsl = resultdaily[resultdaily['msl1'].notna()]

print(np.sqrt(mean_squared_error(resultdailytp['tp1'], resultdailytp['tp'])))
print(np.sqrt(mean_squared_error(resultdailyt2m['t2m1'], resultdailyt2m['t2m'])))
print(np.sqrt(mean_squared_error(resultdailymn2t['mn2t1'], resultdailymn2t['mn2t'])))
print(np.sqrt(mean_squared_error(resultdailymx2t['mx2t1'], resultdailymx2t['mx2t'])))
print(np.sqrt(mean_squared_error(resultdailysd['sd1'], resultdailysd['sdadj'])))
print(np.sqrt(mean_squared_error(resultdailywsp['wspeed1'], resultdailywsp['wspeed'])))
print(np.sqrt(mean_squared_error(resultdailymsl['msl1'], resultdailymsl['msl'])))

In [ ]:
# Soil only - merge the reduced datasets
resultdailyreduced = pd.concat([dfdailyreduced, dfstationsoil], axis=1)
resultdailyreduced = resultdailyreduced[['stl2', '20cm', 'stl3', '40cm', '60cm', '80cm', 'stl4', '160cm', '320cm']]
resultdailyreduced

In [ ]:
# find correlations
print(resultdailyreduced['stl2'].corr(resultdailyreduced['20cm']))
print(resultdailyreduced['stl3'].corr(resultdailyreduced['40cm']))
print(resultdailyreduced['stl3'].corr(resultdailyreduced['60cm']))
print(resultdailyreduced['stl3'].corr(resultdailyreduced['80cm']))
print(resultdailyreduced['stl4'].corr(resultdailyreduced['160cm']))
print(resultdailyreduced['stl4'].corr(resultdailyreduced['320cm']))

In [ ]:
# find variance ratio
print((resultdailyreduced['stl2'].var())/(resultdailyreduced['20cm'].var()))
print((resultdailyreduced['stl3'].var())/(resultdailyreduced['40cm'].var()))
print((resultdailyreduced['stl3'].var())/(resultdailyreduced['60cm'].var()))
print((resultdailyreduced['stl3'].var())/(resultdailyreduced['80cm'].var()))
print((resultdailyreduced['stl4'].var())/(resultdailyreduced['160cm'].var()))
print((resultdailyreduced['stl4'].var())/(resultdailyreduced['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultdailyreduced20 = resultdailyreduced[resultdailyreduced['20cm'].notna()]
resultdailyreduced40 = resultdailyreduced[resultdailyreduced['40cm'].notna()]
resultdailyreduced60 = resultdailyreduced[resultdailyreduced['60cm'].notna()]
resultdailyreduced80 = resultdailyreduced[resultdailyreduced['80cm'].notna()]
resultdailyreduced160 = resultdailyreduced[resultdailyreduced['160cm'].notna()]
resultdailyreduced320 = resultdailyreduced[resultdailyreduced['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultdailyreduced20['20cm'], resultdailyreduced20['stl2'])))
print(np.sqrt(mean_squared_error(resultdailyreduced40['40cm'], resultdailyreduced40['stl3'])))
print(np.sqrt(mean_squared_error(resultdailyreduced60['60cm'], resultdailyreduced60['stl3'])))
print(np.sqrt(mean_squared_error(resultdailyreduced80['80cm'], resultdailyreduced80['stl3'])))
print(np.sqrt(mean_squared_error(resultdailyreduced160['160cm'], resultdailyreduced160['stl4'])))
print(np.sqrt(mean_squared_error(resultdailyreduced320['320cm'], resultdailyreduced320['stl4'])))

## Monthly data

In [ ]:
# Resample daily to monthly
resultmonthly = resultdaily.copy()
#resultmonthly = resultmonthly.drop(['mx2t', 'mn2t', 'mx2t1', 'mn2t1'], axis=1)

resultmonthlymean = resultmonthly[['t2m', 't2m1', 'msl', 'msl1', 'sdadj', 'sd1']].resample('M').mean()
resultmonthlymean = resultmonthlymean.rename(columns={'sdadj':'sdadjmean', 'sd1':'sd1mean'})
resultmonthlysum = resultmonthly[['tp', 'tp1', 'sdadj', 'sd1']].resample('M').sum()
resultmonthlysum = resultmonthlysum.rename(columns={'sdadj':'sdadjsum', 'sd1':'sd1sum'})
resultmonthlymin = resultmonthly[['mn2t', 'mn2t1']].resample('M').min()
resultmonthlymax = resultmonthly[['mx2t', 'mx2t1']].resample('M').max()

resultmonthly = pd.concat([resultmonthlymean, resultmonthlysum, resultmonthlymin, resultmonthlymax], axis=1)

# set months with > 10% missing data to np.nan
#resultmonthly['msl1'].loc['1977-11'] = np.nan
#resultmonthly['tp1'].loc['1977-11'] = np.nan
#resultmonthly['sd1sum'].loc['1977-11'] = np.nan

resultmonthly

In [ ]:
# Find biases
resultmonthly['tpbias'] = resultmonthly['tp1'] - resultmonthly['tp']
tpbias = (resultmonthly['tpbias'].sum())/(resultmonthly['tpbias'].notnull().sum())
resultmonthly['mslbias'] = resultmonthly['msl1'] - resultmonthly['msl']
mslbias = (resultmonthly['mslbias'].sum())/(resultmonthly['mslbias'].notnull().sum())
resultmonthly['t2mbias'] = resultmonthly['t2m1'] - resultmonthly['t2m']
t2mbias = (resultmonthly['t2mbias'].sum())/(resultmonthly['t2mbias'].notnull().sum())
resultmonthly['sdmeanbias'] = resultmonthly['sd1mean'] - resultmonthly['sdadjmean']
sdmeanbias = (resultmonthly['sdmeanbias'].sum())/(resultmonthly['sdmeanbias'].notnull().sum())
resultmonthly['sdsumbias'] = resultmonthly['sd1sum'] - resultmonthly['sdadjsum']
sdsumbias = (resultmonthly['sdsumbias'].sum())/(resultmonthly['sdsumbias'].notnull().sum())
print(tpbias)
print(mslbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultmonthly['tp'].corr(resultmonthly['tp1']))
print(resultmonthly['msl'].corr(resultmonthly['msl1']))
print(resultmonthly['t2m'].corr(resultmonthly['t2m1']))
print(resultmonthly['sdadjmean'].corr(resultmonthly['sd1mean']))
print(resultmonthly['sdadjsum'].corr(resultmonthly['sd1sum']))

In [ ]:
# find variance ratio
print((resultmonthly['tp'].var())/(resultmonthly['tp1'].var()))
print((resultmonthly['msl'].var())/(resultmonthly['msl1'].var()))
print((resultmonthly['t2m'].var())/(resultmonthly['t2m1'].var()))
print((resultmonthly['sdadjmean'].var())/(resultmonthly['sd1mean'].var()))
print((resultmonthly['sdadjsum'].var())/(resultmonthly['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
resultmonthlytp = resultmonthly[resultmonthly['tp1'].notna()]
resultmonthlymsl = resultmonthly[resultmonthly['msl1'].notna()]
resultmonthlyt2m = resultmonthly[resultmonthly['t2m1'].notna()]
resultmonthlysdmean = resultmonthly[resultmonthly['sd1mean'].notna()]
resultmonthlysdsum = resultmonthly[resultmonthly['sd1sum'].notna()]
print(np.sqrt(mean_squared_error(resultmonthlytp['tp1'], resultmonthlytp['tp'])))
print(np.sqrt(mean_squared_error(resultmonthlymsl['msl1'], resultmonthlymsl['msl'])))
print(np.sqrt(mean_squared_error(resultmonthlyt2m['t2m1'], resultmonthlyt2m['t2m'])))
print(np.sqrt(mean_squared_error(resultmonthlysdmean['sd1mean'], resultmonthlysdmean['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultmonthlysdsum['sd1sum'], resultmonthlysdsum['sdadjsum'])))

In [ ]:
# Treat soil separately
resultmonthlyreduced = resultdailyreduced.copy()
resultmonthlyreduced = resultmonthlyreduced.resample('M').mean()

# set months with > 10% missing data to np.nan
resultmonthlyreduced['40cm'].loc['1991-05'] = np.nan
resultmonthlyreduced['160cm'].loc['1991-05', '1991-09', '1992-01', '1993-07'] = np.nan
resultmonthlyreduced['320cm'].loc['1992-06', '1992-07'] = np.nan

resultmonthlyreduced

In [ ]:
# find correlations
print(resultmonthlyreduced['stl2'].corr(resultmonthlyreduced['20cm']))
print(resultmonthlyreduced['stl3'].corr(resultmonthlyreduced['40cm']))
print(resultmonthlyreduced['stl3'].corr(resultmonthlyreduced['60cm']))
print(resultmonthlyreduced['stl3'].corr(resultmonthlyreduced['80cm']))
print(resultmonthlyreduced['stl4'].corr(resultmonthlyreduced['160cm']))
print(resultmonthlyreduced['stl4'].corr(resultmonthlyreduced['320cm']))

In [ ]:
# find variance ratio
print((resultmonthlyreduced['stl2'].var())/(resultmonthlyreduced['20cm'].var()))
print((resultmonthlyreduced['stl3'].var())/(resultmonthlyreduced['40cm'].var()))
print((resultmonthlyreduced['stl3'].var())/(resultmonthlyreduced['60cm'].var()))
print((resultmonthlyreduced['stl3'].var())/(resultmonthlyreduced['80cm'].var()))
print((resultmonthlyreduced['stl4'].var())/(resultmonthlyreduced['160cm'].var()))
print((resultmonthlyreduced['stl4'].var())/(resultmonthlyreduced['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultmonthlyreduced20 = resultmonthlyreduced[resultmonthlyreduced['20cm'].notna()]
resultmonthlyreduced40 = resultmonthlyreduced[resultmonthlyreduced['40cm'].notna()]
resultmonthlyreduced60 = resultmonthlyreduced[resultmonthlyreduced['60cm'].notna()]
resultmonthlyreduced80 = resultmonthlyreduced[resultmonthlyreduced['80cm'].notna()]
resultmonthlyreduced160 = resultmonthlyreduced[resultmonthlyreduced['160cm'].notna()]
resultmonthlyreduced320 = resultmonthlyreduced[resultmonthlyreduced['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultmonthlyreduced20['20cm'], resultmonthlyreduced20['stl2'])))
print(np.sqrt(mean_squared_error(resultmonthlyreduced40['40cm'], resultmonthlyreduced40['stl3'])))
print(np.sqrt(mean_squared_error(resultmonthlyreduced60['60cm'], resultmonthlyreduced60['stl3'])))
print(np.sqrt(mean_squared_error(resultmonthlyreduced80['80cm'], resultmonthlyreduced80['stl3'])))
print(np.sqrt(mean_squared_error(resultmonthlyreduced160['160cm'], resultmonthlyreduced160['stl4'])))
print(np.sqrt(mean_squared_error(resultmonthlyreduced320['320cm'], resultmonthlyreduced320['stl4'])))

## Seasonal data

In [ ]:
# write month_to_season function - RUN EVERY TIME
def month_to_season(x):
    if x == 3 or x == 4 or x == 5:
        season = 'MAM'
    elif x == 6 or x == 7 or x == 8:
        season = 'JJA'
    elif x == 9 or x == 10 or x == 11:
        season = 'SON'
    elif x == 12 or x == 1 or x == 2:
        season = 'DJF'
    else:
        season = np.nan
    return season

In [ ]:
# Process seasonal data from monthly
resultseasonal = resultmonthly.copy()

# apply function to database
resultseasonal['season'] = resultseasonal.index.to_series()
resultseasonal['season'] = resultseasonal['season'].dt.month
resultseasonal['season'] = resultseasonal['season'].apply(month_to_season)

# group by season
resultseasonalmean = resultseasonal.groupby([resultseasonal.index.year, 'season'], sort=False)['t2m', 't2m1', 'msl', 'msl1', 'sdadjmean', 'sd1mean'].mean()
resultseasonalsum = resultseasonal.groupby([resultseasonal.index.year, 'season'], sort=False)['tp', 'tp1', 'sdadjsum', 'sd1sum'].sum()

resultseasonal = pd.concat([resultseasonalmean, resultseasonalsum], axis=1)
resultseasonal

In [ ]:
# find correlations
print(resultseasonal['tp'].corr(resultseasonal['tp1']))
print(resultseasonal['msl'].corr(resultseasonal['msl1']))
print(resultseasonal['t2m'].corr(resultseasonal['t2m1']))
print(resultseasonal['sdadjmean'].corr(resultseasonal['sd1mean']))
print(resultseasonal['sdadjsum'].corr(resultseasonal['sd1sum']))

In [ ]:
# find variance ratio
print((resultseasonal['tp'].var())/(resultseasonal['tp1'].var()))
print((resultseasonal['msl'].var())/(resultseasonal['msl1'].var()))
print((resultseasonal['t2m'].var())/(resultseasonal['t2m1'].var()))
print((resultseasonal['sdadjmean'].var())/(resultseasonal['sd1mean'].var()))
print((resultseasonal['sdadjsum'].var())/(resultseasonal['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
print(np.sqrt(mean_squared_error(resultseasonal['tp1'], resultseasonal['tp'])))
print(np.sqrt(mean_squared_error(resultseasonal['msl1'], resultseasonal['msl'])))
print(np.sqrt(mean_squared_error(resultseasonal['t2m1'], resultseasonal['t2m'])))
print(np.sqrt(mean_squared_error(resultseasonal['sd1mean'], resultseasonal['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultseasonal['sd1sum'], resultseasonal['sdadjsum'])))

In [ ]:
# Treat soil separately
resultseasonalreduced = resultmonthlyreduced.copy()

# apply function to database
resultseasonalreduced['season'] = resultseasonalreduced.index.to_series()
resultseasonalreduced['season'] = resultseasonalreduced['season'].dt.month
resultseasonalreduced['season'] = resultseasonalreduced['season'].apply(month_to_season)

# group by season
resultseasonalreduced = resultseasonalreduced.groupby([resultseasonalreduced.index.year, 'season'], sort=False).mean()

resultseasonalreduced

In [ ]:
# find correlations
print(resultseasonalreduced['stl2'].corr(resultseasonalreduced['20cm']))
print(resultseasonalreduced['stl3'].corr(resultseasonalreduced['40cm']))
print(resultseasonalreduced['stl3'].corr(resultseasonalreduced['60cm']))
print(resultseasonalreduced['stl3'].corr(resultseasonalreduced['80cm']))
print(resultseasonalreduced['stl4'].corr(resultseasonalreduced['160cm']))
print(resultseasonalreduced['stl4'].corr(resultseasonalreduced['320cm']))

In [ ]:
# find variance ratio
print((resultseasonalreduced['stl2'].var())/(resultseasonalreduced['20cm'].var()))
print((resultseasonalreduced['stl3'].var())/(resultseasonalreduced['40cm'].var()))
print((resultseasonalreduced['stl3'].var())/(resultseasonalreduced['60cm'].var()))
print((resultseasonalreduced['stl3'].var())/(resultseasonalreduced['80cm'].var()))
print((resultseasonalreduced['stl4'].var())/(resultseasonalreduced['160cm'].var()))
print((resultseasonalreduced['stl4'].var())/(resultseasonalreduced['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultseasonalreduced20 = resultseasonalreduced[resultseasonalreduced['20cm'].notna()]
resultseasonalreduced40 = resultseasonalreduced[resultseasonalreduced['40cm'].notna()]
resultseasonalreduced60 = resultseasonalreduced[resultseasonalreduced['60cm'].notna()]
resultseasonalreduced80 = resultseasonalreduced[resultseasonalreduced['80cm'].notna()]
resultseasonalreduced160 = resultseasonalreduced[resultseasonalreduced['160cm'].notna()]
resultseasonalreduced320 = resultseasonalreduced[resultseasonalreduced['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultseasonalreduced20['20cm'], resultseasonalreduced20['stl2'])))
print(np.sqrt(mean_squared_error(resultseasonalreduced40['40cm'], resultseasonalreduced40['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduced60['60cm'], resultseasonalreduced60['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduced80['80cm'], resultseasonalreduced80['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduced160['160cm'], resultseasonalreduced160['stl4'])))
print(np.sqrt(mean_squared_error(resultseasonalreduced320['320cm'], resultseasonalreduced320['stl4'])))

## Separated seasonal datasets

### Winter

In [ ]:
resultseasonaldjfwind = resultdaily.copy()
resultseasonaldjfwind = resultseasonaldjfwind[['wspeed', 'wspeed1']]

# apply function to database
resultseasonaldjfwind['season'] = resultseasonaldjfwind.index.to_series()
resultseasonaldjfwind['season'] = resultseasonaldjfwind['season'].dt.month
resultseasonaldjfwind['season'] = resultseasonaldjfwind['season'].apply(month_to_season)

# find only DJF 
resultseasonaldjfwind = resultseasonaldjfwind[resultseasonaldjfwind['season'] == 'DJF']
resultseasonaldjfwind = resultseasonaldjfwind.drop('season', axis=1)
resultseasonaldjfwind = resultseasonaldjfwind['1966-01-01':]
resultseasonaldjfwind

In [ ]:
resultseasonaldjfwind['wspbias'] = resultseasonaldjfwind['wspeed1'] - resultseasonaldjfwind['wspeed']
wspbias = (resultseasonaldjfwind['wspbias'].sum())/(resultseasonaldjfwind['wspbias'].notnull().sum())
wspbias

In [ ]:
print(resultseasonaldjfwind['wspeed'].corr(resultseasonaldjfwind['wspeed1']))
print((resultseasonaldjfwind['wspeed'].var())/(resultseasonaldjfwind['wspeed1'].var()))
print(np.sqrt(mean_squared_error(resultseasonaldjfwind['wspeed1'], resultseasonaldjfwind['wspeed'])))

In [ ]:
resultseasonaldjf = resultmonthly.copy()

# apply function to database
resultseasonaldjf['season'] = resultseasonaldjf.index.to_series()
resultseasonaldjf['season'] = resultseasonaldjf['season'].dt.month
resultseasonaldjf['season'] = resultseasonaldjf['season'].apply(month_to_season)

# find only DJF 
resultseasonaldjf = resultseasonaldjf[resultseasonaldjf['season'] == 'DJF']
resultseasonaldjf = resultseasonaldjf.drop('season', axis=1)
resultseasonaldjf

In [ ]:
resultseasonaldjf['tpbias'] = resultseasonaldjf['tp1'] - resultseasonaldjf['tp']
tpbias = (resultseasonaldjf['tpbias'].sum())/(resultseasonaldjf['tpbias'].notnull().sum())
resultseasonaldjf['mslbias'] = resultseasonaldjf['msl1'] - resultseasonaldjf['msl']
mslbias = (resultseasonaldjf['mslbias'].sum())/(resultseasonaldjf['mslbias'].notnull().sum())
resultseasonaldjf['t2mbias'] = resultseasonaldjf['t2m1'] - resultseasonaldjf['t2m']
t2mbias = (resultseasonaldjf['t2mbias'].sum())/(resultseasonaldjf['t2mbias'].notnull().sum())
resultseasonaldjf['sdmeanbias'] = resultseasonaldjf['sd1mean'] - resultseasonaldjf['sdadjmean']
sdmeanbias = (resultseasonaldjf['sdmeanbias'].sum())/(resultseasonaldjf['sdmeanbias'].notnull().sum())
resultseasonaldjf['sdsumbias'] = resultseasonaldjf['sd1sum'] - resultseasonaldjf['sdadjsum']
sdsumbias = (resultseasonaldjf['sdsumbias'].sum())/(resultseasonaldjf['sdsumbias'].notnull().sum())
print(tpbias)
print(mslbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultseasonaldjf['tp'].corr(resultseasonaldjf['tp1']))
print(resultseasonaldjf['msl'].corr(resultseasonaldjf['msl1']))
print(resultseasonaldjf['t2m'].corr(resultseasonaldjf['t2m1']))
print(resultseasonaldjf['sdadjmean'].corr(resultseasonaldjf['sd1mean']))
print(resultseasonaldjf['sdadjsum'].corr(resultseasonaldjf['sd1sum']))

In [ ]:
# find variance ratio
print((resultseasonaldjf['tp'].var())/(resultseasonaldjf['tp1'].var()))
print((resultseasonaldjf['msl'].var())/(resultseasonaldjf['msl1'].var()))
print((resultseasonaldjf['t2m'].var())/(resultseasonaldjf['t2m1'].var()))
print((resultseasonaldjf['sdadjmean'].var())/(resultseasonaldjf['sd1mean'].var()))
print((resultseasonaldjf['sdadjsum'].var())/(resultseasonaldjf['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
resultseasonaldjfmsl = resultseasonaldjf[resultseasonaldjf['msl1'].notna()]
print(np.sqrt(mean_squared_error(resultseasonaldjf['tp1'], resulte5seasonaldjf['tp'])))
print(np.sqrt(mean_squared_error(resultseasonaldjfmsl['msl1'], resultseasonaldjfmsl['msl'])))
print(np.sqrt(mean_squared_error(resultseasonaldjf['t2m1'], resulte5seasonaldjf['t2m'])))
print(np.sqrt(mean_squared_error(resultseasonaldjf['sd1mean'], resultseasonaldjf['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultseasonaldjf['sd1sum'], resultseasonaldjf['sdadjsum'])))

In [ ]:
resultseasonalreduceddjf = resultmonthlyreduced.copy()

# apply function to database
resultseasonalreduceddjf['season'] = resultseasonalreduceddjf.index.to_series()
resultseasonalreduceddjf['season'] = resultseasonalreduceddjf['season'].dt.month
resultseasonalreduceddjf['season'] = resultseasonalreduceddjf['season'].apply(month_to_season)

# find only DJF 
resultseasonalreduceddjf = resultseasonalreduceddjf[resultseasonalreduceddjf['season'] == 'DJF']
resultseasonalreduceddjf = resultseasonalreduceddjf.drop('season', axis=1)
resultseasonalreduceddjf

In [ ]:
# find correlations
print(resultseasonalreduceddjf['stl2'].corr(resultseasonalreduceddjf['20cm']))
print(resultseasonalreduceddjf['stl3'].corr(resultseasonalreduceddjf['40cm']))
print(resultseasonalreduceddjf['stl3'].corr(resultseasonalreduceddjf['60cm']))
print(resultseasonalreduceddjf['stl3'].corr(resultseasonalreduceddjf['80cm']))
print(resultseasonalreduceddjf['stl4'].corr(resultseasonalreduceddjf['160cm']))
print(resultseasonalreduceddjf['stl4'].corr(resultseasonalreduceddjf['320cm']))

In [ ]:
# find variance ratio
print((resultseasonalreduceddjf['stl2'].var())/(resultseasonalreduceddjf['20cm'].var()))
print((resultseasonalreduceddjf['stl3'].var())/(resultseasonalreduceddjf['40cm'].var()))
print((resultseasonalreduceddjf['stl3'].var())/(resultseasonalreduceddjf['60cm'].var()))
print((resultseasonalreduceddjf['stl3'].var())/(resultseasonalreduceddjf['80cm'].var()))
print((resultseasonalreduceddjf['stl4'].var())/(resultseasonalreduceddjf['160cm'].var()))
print((resultseasonalreduceddjf['stl4'].var())/(resultseasonalreduceddjf['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultseasonalreduceddjf20 = resultseasonalreduceddjf[resultseasonalreduceddjf['20cm'].notna()]
resultseasonalreduceddjf40 = resultseasonalreduceddjf[resultseasonalreduceddjf['40cm'].notna()]
resultseasonalreduceddjf60 = resultseasonalreduceddjf[resultseasonalreduceddjf['60cm'].notna()]
resultseasonalreduceddjf80 = resultseasonalreduceddjf[resultseasonalreduceddjf['80cm'].notna()]
resultseasonalreduceddjf160 = resultseasonalreduceddjf[resultseasonalreduceddjf['160cm'].notna()]
resultseasonalreduceddjf320 = resultseasonalreduceddjf[resultseasonalreduceddjf['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultseasonalreduceddjf20['20cm'], resultseasonalreduceddjf20['stl2'])))
print(np.sqrt(mean_squared_error(resultseasonalreduceddjf40['40cm'], resultseasonalreduceddjf40['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduceddjf60['60cm'], resultseasonalreduceddjf60['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduceddjf80['80cm'], resultseasonalreduceddjf80['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreduceddjf160['160cm'], resultseasonalreduceddjf160['stl4'])))
print(np.sqrt(mean_squared_error(resultseasonalreduceddjf320['320cm'], resultseasonalreduceddjf320['stl4'])))

### Spring

In [ ]:
resultseasonalmamwind = resultdaily.copy()
resultseasonalmamwind = resultseasonalmamwind[['wspeed', 'wspeed1']]

# apply function to database
resultseasonalmamwind['season'] = resultseasonalmamwind.index.to_series()
resultseasonalmamwind['season'] = resultseasonalmamwind['season'].dt.month
resultseasonalmamwind['season'] = resultseasonalmamwind['season'].apply(month_to_season)

# find only MAM
resultseasonalmamwind = resultseasonalmamwind[resultseasonalmamwind['season'] == 'MAM']
resultseasonalmamwind = resultseasonalmamwind.drop('season', axis=1)
resultseasonalmamwind = resultseasonalmamwind['1966-01-01':]
resultseasonalmamwind

In [ ]:
resultseasonalmamwind['wspbias'] = resultseasonalmamwind['wspeed1'] - resultseasonalmamwind['wspeed']
wspbias = (resultseasonalmamwind['wspbias'].sum())/(resultseasonalmamwind['wspbias'].notnull().sum())
wspbias

In [ ]:
print(resultseasonalmamwind['wspeed'].corr(resultseasonalmamwind['wspeed1']))
print((resultseasonalmamwind['wspeed'].var())/(resultseasonalmamwind['wspeed1'].var()))
print(np.sqrt(mean_squared_error(resultseasonalmamwind['wspeed1'], resultseasonalmamwind['wspeed'])))

In [ ]:
resultseasonalmam = resultmonthly.copy()

# apply function to database
resultseasonalmam['season'] = resultseasonalmam.index.to_series()
resultseasonalmam['season'] = resultseasonalmam['season'].dt.month
resultseasonalmam['season'] = resultseasonalmam['season'].apply(month_to_season)

# find only MAM
resultseasonalmam = resultseasonalmam[resultseasonalmam['season'] == 'MAM']
resultseasonalmam = resultseasonalmam.drop('season', axis=1)
resultseasonalmam

In [ ]:
resultseasonalmam['tpbias'] = resultseasonalmam['tp1'] - resultseasonalmam['tp']
tpbias = (resultseasonalmam['tpbias'].sum())/(resultseasonalmam['tpbias'].notnull().sum())
resultseasonalmam['mslbias'] = resultseasonalmam['msl1'] - resultseasonalmam['msl']
mslbias = (resultseasonalmam['mslbias'].sum())/(resultseasonalmam['mslbias'].notnull().sum())
resultseasonalmam['t2mbias'] = resultseasonalmam['t2m1'] - resultseasonalmam['t2m']
t2mbias = (resultseasonalmam['t2mbias'].sum())/(resultseasonalmam['t2mbias'].notnull().sum())
resultseasonalmam['sdmeanbias'] = resultseasonalmam['sd1mean'] - resultseasonalmam['sdadjmean']
sdmeanbias = (resultseasonalmam['sdmeanbias'].sum())/(resultseasonalmam['sdmeanbias'].notnull().sum())
resultseasonalmam['sdsumbias'] = resultseasonalmam['sd1sum'] - resultseasonalmam['sdadjsum']
sdsumbias = (resultseasonalmam['sdsumbias'].sum())/(resultseasonalmam['sdsumbias'].notnull().sum())
print(tpbias)
print(mslbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultseasonalmam['tp'].corr(resultseasonalmam['tp1']))
print(resultseasonalmam['msl'].corr(resultseasonalmam['msl1']))
print(resultseasonalmam['t2m'].corr(resultseasonalmam['t2m1']))
print(resultseasonalmam['sdadjmean'].corr(resultseasonalmam['sd1mean']))
print(resultseasonalmam['sdadjsum'].corr(resultseasonalmam['sd1sum']))

In [ ]:
# find variance ratio
print((resultseasonalmam['tp'].var())/(resultseasonalmam['tp1'].var()))
print((resultseasonalmam['msl'].var())/(resultseasonalmam['msl1'].var()))
print((resultseasonalmam['t2m'].var())/(resultseasonalmam['t2m1'].var()))
print((resultseasonalmam['sdadjmean'].var())/(resultseasonalmam['sd1mean'].var()))
print((resultseasonalmam['sdadjsum'].var())/(resultseasonalmam['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
resultseasonalmammsl = resultseasonalmam[resultseasonalmam['msl1'].notna()]
print(np.sqrt(mean_squared_error(resultseasonalmam['tp1'], resultseasonalmam['tp'])))
print(np.sqrt(mean_squared_error(resulte5seasonalmammsl['msl1'], resultseasonalmammsl['msl'])))
print(np.sqrt(mean_squared_error(resultseasonalmam['t2m1'], resultseasonalmam['t2m'])))
print(np.sqrt(mean_squared_error(resultseasonalmam['sd1mean'], resultseasonalmam['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultseasonalmam['sd1sum'], resultseasonalmam['sdadjsum'])))

In [ ]:
resultseasonalreducedmam = resultmonthlyreduced.copy()

# apply function to database
resultseasonalreducedmam['season'] = resultseasonalreducedmam.index.to_series()
resultseasonalreducedmam['season'] = resultseasonalreducedmam['season'].dt.month
resultseasonalreducedmam['season'] = resultseasonalreducedmam['season'].apply(month_to_season)

# find only MAM
resultseasonalreducedmam = resultseasonalreducedmam[resultseasonalreducedmam['season'] == 'MAM']
resultseasonalreducedmam = resultseasonalreducedmam.drop('season', axis=1)
resultseasonalreducedmam

In [ ]:
# find correlations
print(resultseasonalreducedmam['stl2'].corr(resultseasonalreducedmam['20cm']))
print(resultseasonalreducedmam['stl3'].corr(resultseasonalreducedmam['40cm']))
print(resultseasonalreducedmam['stl3'].corr(resultseasonalreducedmam['60cm']))
print(resultseasonalreducedmam['stl3'].corr(resultseasonalreducedmam['80cm']))
print(resultseasonalreducedmam['stl4'].corr(resultseasonalreducedmam['160cm']))
print(resultseasonalreducedmam['stl4'].corr(resultseasonalreducedmam['320cm']))

In [ ]:
# find variance ratio
print((resultseasonalreducedmam['stl2'].var())/(resultseasonalreducedmam['20cm'].var()))
print((resultseasonalreducedmam['stl3'].var())/(resultseasonalreducedmam['40cm'].var()))
print((resultseasonalreducedmam['stl3'].var())/(resultseasonalreducedmam['60cm'].var()))
print((resultseasonalreducedmam['stl3'].var())/(resultseasonalreducedmam['80cm'].var()))
print((resultseasonalreducedmam['stl4'].var())/(resultseasonalreducedmam['160cm'].var()))
print((resultseasonalreducedmam['stl4'].var())/(resultseasonalreducedmam['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultseasonalreducedmam20 = resultseasonalreducedmam[resultseasonalreducedmam['20cm'].notna()]
resultseasonalreducedmam40 = resultseasonalreducedmam[resultseasonalreducedmam['40cm'].notna()]
resultseasonalreducedmam60 = resultseasonalreducedmam[resultseasonalreducedmam['60cm'].notna()]
resultseasonalreducedmam80 = resultseasonalreducedmam[resultseasonalreducedmam['80cm'].notna()]
resultseasonalreducedmam160 = resultseasonalreducedmam[resultseasonalreducedmam['160cm'].notna()]
resultseasonalreducedmam320 = resultseasonalreducedmam[resultseasonalreducedmam['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultseasonalreducedmam20['20cm'], resultseasonalreducedmam20['stl2'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedmam40['40cm'], resultseasonalreducedmam40['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedmam60['60cm'], resultseasonalreducedmam60['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedmam80['80cm'], resultseasonalreducedmam80['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedmam160['160cm'], resultseasonalreducedmam160['stl4'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedmam320['320cm'], resultseasonalreducedmam320['stl4'])))

### Summer

In [ ]:
resultseasonaljjawind = resultdaily.copy()
resultseasonaljjawind = resultseasonaljjawind[['wspeed', 'wspeed1']]

# apply function to database
resultseasonaljjawind['season'] = resultseasonaljjawind.index.to_series()
resultseasonaljjawind['season'] = resultseasonaljjawind['season'].dt.month
resultseasonaljjawind['season'] = resultseasonaljjawind['season'].apply(month_to_season)

# find only JJA
resultseasonaljjawind = resultseasonaljjawind[resulte5seasonaljjawind['season'] == 'JJA']
resultseasonaljjawind = resultseasonaljjawind.drop('season', axis=1)
resultseasonaljjawind = resultseasonaljjawind['1966-01-01':]
resultseasonaljjawind

In [ ]:
resultseasonaljjawind['wspbias'] = resultseasonaljjawind['wspeed1'] - resultseasonaljjawind['wspeed']
wspbias = (resultseasonaljjawind['wspbias'].sum())/(resultseasonaljjawind['wspbias'].notnull().sum())
wspbias

In [ ]:
print(resultseasonaljjawind['wspeed'].corr(resultseasonaljjawind['wspeed1']))
print((resultseasonaljjawind['wspeed'].var())/(resultseasonaljjawind['wspeed1'].var()))
print(np.sqrt(mean_squared_error(resultseasonaljjawind['wspeed1'], resultseasonaljjawind['wspeed'])))

In [ ]:
resultseasonaljja = resultmonthly.copy()

# apply function to database
resultseasonaljja['season'] = resultseasonaljja.index.to_series()
resultseasonaljja['season'] = resultseasonaljja['season'].dt.month
resultseasonaljja['season'] = resultseasonaljja['season'].apply(month_to_season)

# find only JJA
resultseasonaljja = resultseasonaljja[resultseasonaljja['season'] == 'JJA']
resultseasonaljja = resultseasonaljja.drop('season', axis=1)
resultseasonaljja

In [ ]:
resultseasonaljja['tpbias'] = resultseasonaljja['tp1'] - resultseasonaljja['tp']
tpbias = (resultseasonaljja['tpbias'].sum())/(resultseasonaljja['tpbias'].notnull().sum())
resultseasonaljja['mslbias'] = resultseasonaljja['msl1'] - resultseasonaljja['msl']
mslbias = (resultseasonaljja['mslbias'].sum())/(resultseasonaljja['mslbias'].notnull().sum())
resultseasonaljja['t2mbias'] = resultseasonaljja['t2m1'] - resultseasonaljja['t2m']
t2mbias = (resultseasonaljja['t2mbias'].sum())/(resultseasonaljja['t2mbias'].notnull().sum())
resultseasonaljja['sdmeanbias'] = resultseasonaljja['sd1mean'] - resultseasonaljja['sdadjmean']
sdmeanbias = (resultseasonaljja['sdmeanbias'].sum())/(resultseasonaljja['sdmeanbias'].notnull().sum())
resultseasonaljja['sdsumbias'] = resultseasonaljja['sd1sum'] - resultseasonaljja['sdadjsum']
sdsumbias = (resultseasonaljja['sdsumbias'].sum())/(resultseasonaljja['sdsumbias'].notnull().sum())
print(tpbias)
print(mslbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultseasonaljja['tp'].corr(resultseasonaljja['tp1']))
print(resultseasonaljja['msl'].corr(resultseasonaljja['msl1']))
print(resultseasonaljja['t2m'].corr(resultseasonaljja['t2m1']))
print(resultseasonaljja['sdadjmean'].corr(resultseasonaljja['sd1mean']))
print(resultseasonaljja['sdadjsum'].corr(resultseasonaljja['sd1sum']))

In [ ]:
# find variance ratio
print((resultseasonaljja['tp'].var())/(resultseasonaljja['tp1'].var()))
print((resultseasonaljja['msl'].var())/(resultseasonaljja['msl1'].var()))
print((resultseasonaljja['t2m'].var())/(resultseasonaljja['t2m1'].var()))
print((resultseasonaljja['sdadjmean'].var())/(resultseasonaljja['sd1mean'].var()))
print((resultseasonaljja['sdadjsum'].var())/(resultseasonaljja['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
resultseasonaljjamsl = resulte5seasonaljja[resulte5seasonaljja['msl1'].notna()]
print(np.sqrt(mean_squared_error(resultseasonaljja['tp1'], resultseasonaljja['tp'])))
print(np.sqrt(mean_squared_error(resulte5seasonaljjamsl['msl1'], resultseasonaljjamsl['msl'])))
print(np.sqrt(mean_squared_error(resultseasonaljja['t2m1'], resultseasonaljja['t2m'])))
print(np.sqrt(mean_squared_error(resultseasonaljja['sd1mean'], resultseasonaljja['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultseasonaljja['sd1sum'], resultseasonaljja['sdadjsum'])))

In [ ]:
resultseasonalreducedjja = resultmonthlyreduced.copy()

# apply function to database
resultseasonalreducedjja['season'] = resultseasonalreducedjja.index.to_series()
resultseasonalreducedjja['season'] = resultseasonalreducedjja['season'].dt.month
resultseasonalreducedjja['season'] = resultseasonalreducedjja['season'].apply(month_to_season)

# find only JJA
resultseasonalreducedjja = resultseasonalreducedjja[resultseasonalreducedjja['season'] == 'JJA']
resultseasonalreducedjja = resultseasonalreducedjja.drop('season', axis=1)
resultseasonalreducedjja

In [ ]:
# find correlations
print(resultseasonalreducedjja['stl2'].corr(resultseasonalreducedjja['20cm']))
print(resultseasonalreducedjja['stl3'].corr(resultseasonalreducedjja['40cm']))
print(resultseasonalreducedjja['stl3'].corr(resultseasonalreducedjja['60cm']))
print(resultseasonalreducedjja['stl3'].corr(resultseasonalreducedjja['80cm']))
print(resultseasonalreducedjja['stl4'].corr(resultseasonalreducedjja['160cm']))
print(resultseasonalreducedjja['stl4'].corr(resultseasonalreducedjja['320cm']))

In [ ]:
# find variance ratio
print((resultseasonalreducedjja['stl2'].var())/(resultseasonalreducedjja['20cm'].var()))
print((resultseasonalreducedjja['stl3'].var())/(resultseasonalreducedjja['40cm'].var()))
print((resultseasonalreducedjja['stl3'].var())/(resultseasonalreducedjja['60cm'].var()))
print((resultseasonalreducedjja['stl3'].var())/(resultseasonalreducedjja['80cm'].var()))
print((resultseasonalreducedjja['stl4'].var())/(resultseasonalreducedjja['160cm'].var()))
print((resultseasonalreducedjja['stl4'].var())/(resultseasonalreducedjja['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultseasonalreducedjja20 = resultseasonalreducedjja[resultseasonalreducedjja['20cm'].notna()]
resultseasonalreducedjja40 = resultseasonalreducedjja[resultseasonalreducedjja['40cm'].notna()]
resultseasonalreducedjja60 = resultseasonalreducedjja[resultseasonalreducedjja['60cm'].notna()]
resultseasonalreducedjja80 = resultseasonalreducedjja[resultseasonalreducedjja['80cm'].notna()]
resultseasonalreducedjja160 = resultseasonalreducedjja[resultseasonalreducedjja['160cm'].notna()]
resultseasonalreducedjja320 = resultseasonalreducedjja[resultseasonalreducedjja['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultseasonalreducedjja20['20cm'], resultseasonalreducedjja20['stl2'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedjja40['40cm'], resultseasonalreducedjja40['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedjja60['60cm'], resultseasonalreducedjja60['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedjja80['80cm'], resultseasonalreducedjja80['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedjja160['160cm'], resultseasonalreducedjja160['stl4'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedjja320['320cm'], resultseasonalreducedjja320['stl4'])))

### Autumn

In [ ]:
resultseasonalsonwind = resultdaily.copy()
resultseasonalsonwind = resultseasonalsonwind[['wspeed', 'wspeed1']]

# apply function to database
resultseasonalsonwind['season'] = resultseasonalsonwind.index.to_series()
resultseasonalsonwind['season'] = resultseasonalsonwind['season'].dt.month
resultseasonalsonwind['season'] = resultseasonalsonwind['season'].apply(month_to_season)

# find only SON
resultseasonalsonwind = resultseasonalsonwind[resultseasonalsonwind['season'] == 'SON']
resultseasonalsonwind = resultseasonalsonwind.drop('season', axis=1)
resultseasonalsonwind = resultseasonalsonwind['1966-01-01':]
resultseasonalsonwind

In [ ]:
resultseasonalsonwind['wspbias'] = resultseasonalsonwind['wspeed1'] - resultseasonalsonwind['wspeed']
wspbias = (resultseasonalsonwind['wspbias'].sum())/(resultseasonalsonwind['wspbias'].notnull().sum())
wspbias

In [ ]:
resultseasonalsonwindrmse = resultseasonalsonwind[resultseasonalsonwind['wspeed1'].notna()]
print(resultseasonalsonwind['wspeed'].corr(resultseasonalsonwind['wspeed1']))
print((resultseasonalsonwind['wspeed'].var())/(resultseasonalsonwind['wspeed1'].var()))
print(np.sqrt(mean_squared_error(resultseasonalsonwindrmse['wspeed1'], resultseasonalsonwindrmse['wspeed'])))

In [ ]:
resultseasonalson = resultmonthly.copy()

# apply function to database
resultseasonalson['season'] = resultseasonalson.index.to_series()
resultseasonalson['season'] = resultseasonalson['season'].dt.month
resultseasonalson['season'] = resultseasonalson['season'].apply(month_to_season)

# find only SON
resultseasonalson = resultseasonalson[resultseasonalson['season'] == 'SON']
resultseasonalson = resultseasonalson.drop('season', axis=1)
resultseasonalson

In [ ]:
resultseasonalson['tpbias'] = resultseasonalson['tp1'] - resultseasonalson['tp']
tpbias = (resultseasonalson['tpbias'].sum())/(resultseasonalson['tpbias'].notnull().sum())
resultseasonalson['mslbias'] = resultseasonalson['msl1'] - resultseasonalson['msl']
mslbias = (resultseasonalson['mslbias'].sum())/(resultseasonalson['mslbias'].notnull().sum())
resultseasonalson['t2mbias'] = resultseasonalson['t2m1'] - resultseasonalson['t2m']
t2mbias = (resultseasonalson['t2mbias'].sum())/(resultseasonalson['t2mbias'].notnull().sum())
resultseasonalson['sdmeanbias'] = resultseasonalson['sd1mean'] - resultseasonalson['sdadjmean']
sdmeanbias = (resultseasonalson['sdmeanbias'].sum())/(resultseasonalson['sdmeanbias'].notnull().sum())
resultseasonalson['sdsumbias'] = resultseasonalson['sd1sum'] - resultseasonalson['sdadjsum']
sdsumbias = (resultseasonalson['sdsumbias'].sum())/(resultseasonalson['sdsumbias'].notnull().sum())
print(tpbias)
print(mslbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultseasonalson['tp'].corr(resultseasonalson['tp1']))
print(resultseasonalson['msl'].corr(resultseasonalson['msl1']))
print(resultseasonalson['t2m'].corr(resultseasonalson['t2m1']))
print(resultseasonalson['sdadjmean'].corr(resultseasonalson['sd1mean']))
print(resultseasonalson['sdadjsum'].corr(resultseasonalson['sd1sum']))

In [ ]:
# find variance ratio
print((resultseasonalson['tp'].var())/(resultseasonalson['tp1'].var()))
print((resultseasonalson['msl'].var())/(resultseasonalson['msl1'].var()))
print((resultseasonalson['t2m'].var())/(resultseasonalson['t2m1'].var()))
print((resultseasonalson['sdadjmean'].var())/(resultseasonalson['sd1mean'].var()))
print((resultseasonalson['sdadjsum'].var())/(resultseasonalson['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
resultseasonalsontp = resultseasonalson[resultseasonalson['tp1'].notna()]
resultseasonalsonmsl = resultseasonalson[resultseasonalson['msl1'].notna()]
resultseasonalsont2m = resultseasonalson[resultseasonalson['t2m1'].notna()]
resultseasonalsonsdmean = resultseasonalson[resultseasonalson['sd1mean'].notna()]
resultseasonalsonsdsum = resultseasonalson[resultseasonalson['sd1sum'].notna()]
print(np.sqrt(mean_squared_error(resultseasonalsontp['tp1'], resultseasonalsontp['tp'])))
print(np.sqrt(mean_squared_error(resultseasonalsonmsl['msl1'], resultseasonalsonmsl['msl'])))
print(np.sqrt(mean_squared_error(resultseasonalsont2m['t2m1'], resultseasonalsont2m['t2m'])))
print(np.sqrt(mean_squared_error(resultseasonalsonsdmean['sd1mean'], resultseasonalsonsdmean['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultseasonalsonsdsum['sd1sum'], resultseasonalsonsdsum['sdadjsum'])))

In [ ]:
resultseasonalreducedson = resultmonthlyreduced.copy()

# apply function to database
resultseasonalreducedson['season'] = resultseasonalreducedson.index.to_series()
resultseasonalreducedson['season'] = resultseasonalreducedson['season'].dt.month
resultseasonalreducedson['season'] = resultseasonalreducedson['season'].apply(month_to_season)

# find only SON
resultseasonalreducedson = resultseasonalreducedson[resultseasonalreducedson['season'] == 'SON']
resultseasonalreducedson = resultseasonalreducedson.drop('season', axis=1)
resultseasonalreducedson

In [ ]:
# find correlations
print(resultseasonalreducedson['stl2'].corr(resultseasonalreducedson['20cm']))
print(resultseasonalreducedson['stl3'].corr(resultseasonalreducedson['40cm']))
print(resultseasonalreducedson['stl3'].corr(resultseasonalreducedson['60cm']))
print(resultseasonalreducedson['stl3'].corr(resultseasonalreducedson['80cm']))
print(resultseasonalreducedson['stl4'].corr(resultseasonalreducedson['160cm']))
print(resultseasonalreducedson['stl4'].corr(resultseasonalreducedson['320cm']))

In [ ]:
# find variance ratio
print((resultseasonalreducedson['stl2'].var())/(resultseasonalreducedson['20cm'].var()))
print((resultseasonalreducedson['stl3'].var())/(resultseasonalreducedson['40cm'].var()))
print((resultseasonalreducedson['stl3'].var())/(resultseasonalreducedson['60cm'].var()))
print((resultseasonalreducedson['stl3'].var())/(resultseasonalreducedson['80cm'].var()))
print((resultseasonalreducedson['stl4'].var())/(resultseasonalreducedson['160cm'].var()))
print((resultseasonalreducedson['stl4'].var())/(resultseasonalreducedson['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultseasonalreducedson20 = resultseasonalreducedson[resultseasonalreducedson['20cm'].notna()]
resultseasonalreducedson40 = resultseasonalreducedson[resultseasonalreducedson['40cm'].notna()]
resultseasonalreducedson60 = resultseasonalreducedson[resultseasonalreducedson['60cm'].notna()]
resultseasonalreducedson80 = resultseasonalreducedson[resultseasonalreducedson['80cm'].notna()]
resultseasonalreducedson160 = resultseasonalreducedson[resultseasonalreducedson['160cm'].notna()]
resultseasonalreducedson320 = resultseasonalreducedson[resultseasonalreducedson['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultseasonalreducedson20['20cm'], resultseasonalreducedson20['stl2'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedson40['40cm'], resultseasonalreducedson40['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedson60['60cm'], resultseasonalreducedson60['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedson80['80cm'], resultseasonalreducedson80['stl3'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedson160['160cm'], resultseasonalreducedson160['stl4'])))
print(np.sqrt(mean_squared_error(resultseasonalreducedson320['320cm'], resultseasonalreducedson320['stl4'])))

## Annual data

### The long way round - run each time

In [ ]:
# Compile yearly from monthly
# 1979-2020
resultyearly = resultmonthly.copy()
resultyearlymean = resultyearly[['t2m', 't2m1', 'sdadjmean', 'sd1mean']].resample('Y').mean()
resultyearlysum = resultyearly[['tp', 'tp1', 'sdadjsum', 'sd1sum']].resample('Y').sum()
resultyearly = pd.concat([resultyearlymean, resultyearlysum], axis=1)
resultyearly[['t2mcorr', 'tpcorr', 'sdmeancorr', 'sdsumcorr', 't2mvr', 'tpvr', 'sdmeanvr', 'sdsumvr', 't2mrmse', 'tprmse', 'sdmeanrmse', 'sdsumrmse']] = np.nan

resultyearlyt2m = resultmonthly[resultmonthly['t2m1'].notna()]
resultyearlytp = resultmonthly[resultmonthly['tp1'].notna()]
resultyearlysdmean = resultmonthly[resultmonthly['sd1mean'].notna()]
resultyearlysdsum = resultmonthly[resultmonthly['sd1sum'].notna()]

for year in range(1959, 2021):
    try:
        local = resultmonthly.loc[str(year)]

        resultyearly['t2mcorr'].loc[str(year)] = local['t2m'].corr(local['t2m1'])
        resultyearly['tpcorr'].loc[str(year)] = local['tp'].corr(local['tp1'])
        resultyearly['sdmeancorr'].loc[str(year)] = local['sdadjmean'].corr(local['sd1mean'])
        resultyearly['sdsumcorr'].loc[str(year)] = local['sdadjsum'].corr(local['sd1sum'])

        resultyearly['t2mvr'].loc[str(year)] = local['t2m'].var() / local['t2m1'].var()
        resultyearly['tpvr'].loc[str(year)] = local['tp'].var() / local['tp1'].var()
        resultyearly['sdmeanvr'].loc[str(year)] = local['sdadjmean'].var() / local['sd1mean'].var()
        resultyearly['sdsumvr'].loc[str(year)] = local['sdadjsum'].var() / local['sd1sum'].var()

        resultyearly['t2mrmse'].loc[str(year)] = np.sqrt(mean_squared_error(local['t2m1'], local['t2m']))
        resultyearly['tprmse'].loc[str(year)] = np.sqrt(mean_squared_error(local['tp1'], local['tp']))
        resultyearly['sdmeanrmse'].loc[str(year)] = np.sqrt(mean_squared_error(local['sd1mean'], local['sdadjmean']))
        resultyearly['sdsumrmse'].loc[str(year)] = np.sqrt(mean_squared_error(local['sd1sum'], local['sdadjsum']))

    except (KeyError, ValueError, TypeError) as e:
        print(f"Skipping year {year} due to missing data or error: {e}")
        continue

In [ ]:
# Treat soil separately
resultyearlyreduced = resultmonthlyreduced.copy()
resultyearlyreduced = resultyearlyreduced.resample('Y').mean()
resultyearlyreduced[['20corr', '40corr', '60corr', '80corr', '160corr', '320corr', '20vr', '40vr', '60vr', '80vr', '160vr', '320vr', '20rmse', '40rmse', '60rmse', '80rmse', '160rmse', '320rmse']] = np.nan

resultyearlyreduced20 = resultyearlyreduced[resultyearlyreduced['20cm'].notna()]
resultyearlyreduced40 = resultyearlyreduced[resultyearlyreduced['40cm'].notna()]
resultyearlyreduced60 = resultyearlyreduced[resultyearlyreduced['60cm'].notna()]
resultyearlyreduced80 = resultyearlyreduced[resultyearlyreduced['80cm'].notna()]
resultyearlyreduced160 = resultyearlyreduced[resultyearlyreduced['160cm'].notna()]
resultyearlyreduced320 = resultyearlyreduced[resultyearlyreduced['320cm'].notna()]

# Handle missing data
#resultyearlyreduced = resultyearlyreduced.drop(['1997'])
#resultyearlyreduced['160cm'].loc['1991', '1995'] = np.nan

for year in range(1979, 2021):
    try:
        local = resultmonthlyreduced.loc[str(year)]

        resultyearlyreduced['20corr'].loc[str(year)] = local['stl2'].corr(local['20cm'])
        resultyearlyreduced['40corr'].loc[str(year)] = local['stl3'].corr(local['40cm'])
        resultyearlyreduced['60corr'].loc[str(year)] = local['stl3'].corr(local['60cm'])
        resultyearlyreduced['80corr'].loc[str(year)] = local['stl3'].corr(local['80cm'])
        resultyearlyreduced['160corr'].loc[str(year)] = local['stl4'].corr(local['160cm'])
        resultyearlyreduced['320corr'].loc[str(year)] = local['stl4'].corr(local['320cm'])
        
        resultyearlyreduced['20vr'].loc[str(year)] = local['stl2'].var() / local['20cm'].var()
        resultyearlyreduced['40vr'].loc[str(year)] = local['stl3'].var() / local['40cm'].var()
        resultyearlyreduced['60vr'].loc[str(year)] = local['stl3'].var() / local['60cm'].var()
        resultyearlyreduced['80vr'].loc[str(year)] = local['stl3'].var() / local['80cm'].var()
        resultyearlyreduced['160vr'].loc[str(year)] = local['stl4'].var() / local['160cm'].var()
        resultyearlyreduced['320vr'].loc[str(year)] = local['stl4'].var() / local['320cm'].var()
        
        resultyearlyreduced['20rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced20['20cm'].loc[str(year)], resultmonthlyreduced20['stl2'].loc[str(year)]))
        resultyearlyreduced['40rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced40['40cm'].loc[str(year)], resultmonthlyreduced40['stl3'].loc[str(year)]))
        resultyearlyreduced['60rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced60['60cm'].loc[str(year)], resultmonthlyreduced60['stl3'].loc[str(year)]))
        resultyearlyreduced['80rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced80['80cm'].loc[str(year)], resultmonthlyreduced80['stl3'].loc[str(year)]))
        resultyearlyreduced['160rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced160['160cm'].loc[str(year)], resultmonthlyreduced160['stl4'].loc[str(year)]))
        resultyearlyreduced['320rmse'].loc[str(year)] = np.sqrt(mean_squared_error(resultmonthlyreduced320['320cm'].loc[str(year)], resultmonthlyreduced320['stl4'].loc[str(year)]))
        
    except (KeyError, ValueError, TypeError) as e:
        print(f"Skipping year {year} due to missing data or error: {e}")
        continue

### Processed

In [ ]:
resultyearly

In [ ]:
resultyearly['tpbias'] = resultyearly['tp1'] - resultyearly['tp']
tpbias = (resultyearly['tpbias'].sum())/(resultyearly['tpbias'].notnull().sum())
resultyearly['t2mbias'] = resultyearly['t2m1'] - resultyearly['t2m']
t2mbias = (resultyearly['t2mbias'].sum())/(resultyearly['t2mbias'].notnull().sum())
resultyearly['sdmeanbias'] = resultyearly['sd1mean'] - resultyearly['sdadjmean']
sdmeanbias = (resultyearly['sdmeanbias'].sum())/(resultyearly['sdmeanbias'].notnull().sum())
resultyearly['sdsumbias'] = resultyearly['sd1sum'] - resultyearly['sdadjsum']
sdsumbias = (resultyearly['sdsumbias'].sum())/(resultyearly['sdsumbias'].notnull().sum())
print(tpbias)
print(t2mbias)
print(sdmeanbias)
print(sdsumbias)

In [ ]:
# find correlations
print(resultyearly['tp'].corr(resultyearly['tp1']))
print(resultyearly['t2m'].corr(resultyearly['t2m1']))
print(resultyearly['sdadjmean'].corr(resultyearly['sd1mean']))
print(resultyearly['sdadjsum'].corr(resultyearly['sd1sum']))

In [ ]:
# find variance ratio
print((resultyearly['tp'].var())/(resultyearly['tp1'].var()))
print((resultyearly['t2m'].var())/(resultyearly['t2m1'].var()))
print((resultyearly['sdadjmean'].var())/(resultyearly['sd1mean'].var()))
print((resultyearly['sdadjsum'].var())/(resultyearly['sd1sum'].var()))

In [ ]:
# find rmse - actual then predicted
print(np.sqrt(mean_squared_error(resultyearly['tp1'], resultyearly['tp'])))
print(np.sqrt(mean_squared_error(resultyearly['t2m1'], resultyearly['t2m'])))
print(np.sqrt(mean_squared_error(resultyearly['sd1mean'], resultyearly['sdadjmean'])))
print(np.sqrt(mean_squared_error(resultyearly['sd1sum'], resultyearly['sdadjsum'])))

In [ ]:
resultyearlyreduced

In [ ]:
# find correlations
print(resultyearlyreduced['stl2'].corr(resultyearlyreduced['20cm']))
print(resultyearlyreduced['stl3'].corr(resultyearlyreduced['40cm']))
print(resultyearlyreduced['stl3'].corr(resultyearlyreduced['60cm']))
print(resultyearlyreduced['stl3'].corr(resultyearlyreduced['80cm']))
print(resultyearlyreduced['stl4'].corr(resultyearlyreduced['160cm']))
print(resultyearlyreduced['stl4'].corr(resultyearlyreduced['320cm']))

In [ ]:
# find variance ratio
print((resultyearlyreduced['stl2'].var())/(resultyearlyreduced['20cm'].var()))
print((resultyearlyreduced['stl3'].var())/(resultyearlyreduced['40cm'].var()))
print((resultyearlyreduced['stl3'].var())/(resultyearlyreduced['60cm'].var()))
print((resultyearlyreduced['stl3'].var())/(resultyearlyreduced['80cm'].var()))
print((resultyearlyreduced['stl4'].var())/(resultyearlyreduced['160cm'].var()))
print((resultyearlyreduced['stl4'].var())/(resultyearlyreduced['320cm'].var()))

In [ ]:
# find rmse - actual then predicted, using redacted datasets
resultyearlyreduced20 = resultyearlyreduced[resultyearlyreduced['20cm'].notna()]
resultyearlyreduced40 = resultyearlyreduced[resultyearlyreduced['40cm'].notna()]
resultyearlyreduced60 = resultyearlyreduced[resultyearlyreduced['60cm'].notna()]
resultyearlyreduced80 = resultyearlyreduced[resultyearlyreduced['80cm'].notna()]
resultyearlyreduced160 = resultyearlyreduced[resultyearlyreduced['160cm'].notna()]
resultyearlyreduced320 = resultyearlyreduced[resultyearlyreduced['320cm'].notna()]

print(np.sqrt(mean_squared_error(resultyearlyreduced20['20cm'], resultyearlyreduced20['stl2'])))
print(np.sqrt(mean_squared_error(resultyearlyreduced40['40cm'], resultyearlyreduced40['stl3'])))
print(np.sqrt(mean_squared_error(resultyearlyreduced60['60cm'], resultyearlyreduced60['stl3'])))
print(np.sqrt(mean_squared_error(resultyearlyreduced80['80cm'], resultyearlyreduced80['stl3'])))
print(np.sqrt(mean_squared_error(resultyearlyreduced160['160cm'], resultyearlyreduced160['stl4'])))
print(np.sqrt(mean_squared_error(resultyearlyreduced320['320cm'], resultyearlyreduced320['stl4'])))

## Decadal data

In [ ]:
# Resample to decadal from yearly
# 'time' = end year
resultdecadal = resultyearly.copy()
resultdecadal = resultdecadal.rolling(10).mean()
resultdecadal = resultdecadal.drop(resultdecadal.index[[0, 1, 2, 3, 4, 5, 6, 7, 8]])
resultdecadal

In [ ]:
# Treat soil separately
# 'time' = end year
resultdecadalreduced = resultyearlyreduced.copy()
resultdecadalreduced = resultdecadalreduced.rolling(10, min_periods=8).mean()
resultdecadalreduced = resultdecadalreduced.drop(resultdecadalreduced.index[[0, 1, 2, 3, 4, 5, 6, 7, 8]])
resultdecadalreduced

# Plots

In [ ]:
# Raw annual values
# Edit for each variable
data1 = resultyearly.assign(Dataset='ERA5')
#data2 = resultyearly.assign(Dataset='ERA5-land')
data3 = resultyearly.assign(Dataset='met station')

#f = plt.figure(figsize=[4,6])
ax = f.add_subplot(111)

#cdf = pd.concat([data1, data2, data3])    
cdf = pd.concat([data1, data3])    
mdf = pd.melt(cdf, id_vars=['Dataset'], var_name=['Variable'])

mdf = mdf[(mdf['Variable'] == 'sdadjmean') | (mdf['Variable'] == 'sd1mean')]

ax = data1.plot(y='sdadjmean', use_index=True, label='ERA5')
#data2.plot(ax=ax, y='sdadjmean', use_index=True, color='red', label='ERA5-land')
data3.plot(ax=ax, y='sd1mean', use_index=True, color='green', label='station')

#plt.ylim([7500, 23500])
#ax.get_legend().remove()
#plt.axhline(y=1, color='g', linestyle='--')

plt.title(f'{station} snow depth yearly mean 1959-2020')
plt.xlabel('year')
plt.ylabel('snow depth yearly mean (cm)')

#plt.savefig(f'{station}sd.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation plots
# Edit for each variable
data1 = resultdecadal.assign(Dataset='ERA5')
#data2 = resultdecadal.assign(Dataset='ERA5-land')

#f = plt.figure(figsize=[4,6])
ax = f.add_subplot(111)

#cdf = pd.concat([data1, data2])    
cdf = pd.concat([data1])    
mdf = pd.melt(cdf, id_vars=['Dataset'], var_name=['Variable'])

mdf = mdf[(mdf['Variable'] == 'sdmeancorr')]

ax = data1.plot(y='sdmeancorr', use_index=True, label='ERA5 correlation')
#ax = data2.plot(y='sdmeancorr', use_index=True, label='ERA5-Land correlation')

#plt.ylim([7500, 23500])
ax.get_legend().remove()
plt.axhline(y=1, color='g', linestyle='--')

plt.title(f'{station} decadal snow depth correlation 1959-2020')
plt.xlabel('End year of decade')

#plt.savefig(f'{station}sdcorr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Seasonal boxplot
# Edit for each variable
data1 = resultseasonaldjf.assign(Season='Winter')
data2 = resultseasonalmam.assign(Season='Spring')
data3 = resultseasonaljja.assign(Season='Summer')
data4 = resultseasonalson.assign(Season='Autumn')

#f = plt.figure(figsize=[4,6])
ax = f.add_subplot(111)

cdf = pd.concat([data1, data2, data3, data4])    
mdf = pd.melt(cdf, id_vars=['Season'], var_name=['Dataset'])

mdf = mdf[(mdf['Dataset'] == 'sdadjmean') | (mdf['Dataset'] == 'sd1mean')]
ax = sns.boxplot(x="Season", y="value", hue="Dataset", data=mdf)

#plt.ylim([-0.5, 10.9])
#ax.get_legend().remove()

plt.title(f'{station} seasonal snow depth boxplot 1959-2020')
plt.ylabel('Average snow depth (cm)')

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ['ERA5', 'Station'])

#plt.savefig(f'{station}sdseas.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# basic era5 vs met station obvs
# set day/month/year/season to compare
resulttime = result.copy()
resulttime = resulttime.loc['2000':'2001']
#resulttime.plot('t2m', 't2m1', kind='scatter')

#sns.regplot(x='stl4', y='320cm', data=resultyearlyreduced, ci=95, line_kws={'color':'red'})
#resultdecadal.plot(y='sdcorr', use_index=True)

fig = plt.figure()
ax1 = fig.add_subplot(111)
pd.plotting.register_matplotlib_converters()
#resultdecadal.index = resultdecadal.index.to_timestamp()

ax1.scatter(x=resultdecadal.index, y=resultdecadal.t2mcorr, c='b', label='era5')
plt.legend(loc='upper left');
plt.show()

In [ ]:
# plot multiple columns in one graph
resulttime.plot(use_index=True, y=['t2m', 'd2m'], color=['blue', 'orange'])
plt.show()

In [ ]:
# boxplot
resultdecadalreduced.boxplot(column=['20rmse', '40rmse', '60rmse', '80rmse', '160rmse', '320rmse'])
plt.show()

In [ ]:
# basic correlation heatmap
corr = resultmonthlyreduced.corr()
corr.style.background_gradient(cmap='coolwarm')